In [1]:
!sudo apt-get install apt-transport-https ca-certificates dirmngr
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv E0C56BD4

!echo "deb https://repo.clickhouse.tech/deb/stable/ main/" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
!sudo apt-get update

!sudo apt-get install -y clickhouse-server clickhouse-client

!sudo service clickhouse-server start
!clickhouse-client

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20210119~18.04.1).
dirmngr is already the newest version (2.2.4-1ubuntu1.4).
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 1,696 B of archives.
After this operation, 153 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 apt-transport-https all 1.6.12ubuntu0.2 [1,696 B]
Fetched 1,696 B in 0s (9,068 B/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend:

In [2]:
!pip install clickhouse-driver

     |████████████████████████████████| 481kB 5.2MB/s 


In [61]:
from clickhouse_driver import Client
client = Client(host='localhost')
client.execute('SHOW DATABASES')

[('default',), ('system',)]

In [62]:
import pandas as pd
import numpy as np
from datetime import datetime

In [63]:
dateparser = lambda x: datetime.strptime(x, "%d.%m.%Y")
df = pd.read_csv('data.csv',sep=';',decimal=',',parse_dates = ['date'], date_parser=dateparser)

In [64]:
print(df.head())
print(df.info())

        date   city   manager   product  amount
0 2020-01-01  city1  manager1  product1      20
1 2020-01-02  city2  manager2  product2       6
2 2020-01-03  city1  manager3  product3       7
3 2020-01-04  city2  manager4  product4      10
4 2020-01-05  city1  manager1  product5      12
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     20 non-null     datetime64[ns]
 1   city     20 non-null     object        
 2   manager  20 non-null     object        
 3   product  20 non-null     object        
 4   amount   20 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 928.0+ bytes
None


In [65]:
client.execute('DROP DATABASE IF EXISTS db')

[]

In [66]:
client.execute('CREATE DATABASE db')

[]

In [67]:
client.execute('SHOW DATABASES')

[('db',), ('default',), ('system',)]

In [68]:
client = Client(host='localhost', user='default', port='9000', database='db',settings={'columnar': True})

In [70]:
client.execute('DROP TABLE IF EXISTS test')

[]

In [71]:
client.execute('CREATE TABLE test (date Date, city String, manager String, product String, amount Int32) ENGINE = Memory')

[]

In [72]:
client.execute('SHOW TABLES FROM db')

[('test',)]

In [73]:
client.execute("INSERT INTO test VALUES", df.to_dict('records'))

20

In [74]:
def select_clickhouse(sql):
  return client.query_dataframe(sql)

In [109]:
sql = '''SELECT * 
         FROM test as t 
         ORDER BY t.city, t.manager, t.product
         LIMIT 25'''

In [110]:
select_clickhouse(sql)

,date,city,manager,product,amount
0,2020-01-01,city1,manager1,product1,20
1,2020-01-09,city1,manager1,product2,23
2,2020-01-17,city1,manager1,product3,6
3,2020-01-05,city1,manager1,product5,12
4,2020-01-13,city1,manager1,product6,22
5,2020-01-15,city1,manager3,product1,12
6,2020-01-03,city1,manager3,product3,7
7,2020-01-11,city1,manager3,product4,21
8,2020-01-19,city1,manager3,product5,9
9,2020-01-07,city1,manager3,product7,3


### Справка по оконным функциям

In [ ]:
# https://clickhouse.tech/docs/en/sql-reference/window-functions/

In [150]:
client.execute('set allow_experimental_window_functions = 1;')

[]

In [151]:
sql = '''SELECT 
                date,
                city, 
                manager,
                amount,
                sum(amount) over w as amount_cum_sum
         FROM test 
         WINDOW w AS
         (PARTITION BY city,manager
         ORDER BY date,city, manager
         ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW);'''

In [152]:
select_clickhouse(sql)

,date,city,manager,amount,amount_cum_sum
0,2020-01-01,city1,manager1,20,20
1,2020-01-05,city1,manager1,12,32
2,2020-01-09,city1,manager1,23,55
3,2020-01-13,city1,manager1,22,77
4,2020-01-17,city1,manager1,6,83
5,2020-01-03,city1,manager3,7,7
6,2020-01-07,city1,manager3,3,10
7,2020-01-11,city1,manager3,21,31
8,2020-01-15,city1,manager3,12,43
9,2020-01-19,city1,manager3,9,52
